In [1]:
from importlib import reload
import logging
logging.basicConfig(level=logging.INFO)
logging.info("HEllo!")
import thesispipeline
reload(thesispipeline)
ThesisPipeline = thesispipeline.ThesisPipeline

INFO:root:HEllo!
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes

## Topology graph

### Graph Convolutional Network

In [ ]:
thesis = ThesisPipeline()
thesis.load_data()
thesis.make_protein_groups()
thesis.check_protein_groups()
thesis.remove_interior_nodes()
thesis.make_features()
thesis.make_adjacency_matrices()
thesis.make_targets()
thesis.convert_matrices_32bits()
thesis.filter_by_node_amount(amount=1000)
thesis.make_positive_weight()
thesis.make_laplacians()
thesis.pad_matrices()
thesis.make_masks()
thesis.run_cv_gcn(epochs=40)

### Local GCN

For starters, NeighborhoodIterator requires to have the graph instance at training time. So we'll need to keep track of how features and graphs match (i.e. also filter graphs in ThPi.filter_by_node_amount)

To implement it in a new class would be ideal. I should rewrite the parent class so polymorphism can be achievied easily.  Writing a private method `_train_step` would suit this. This also requires wrapping `fit_cv_groups` and `fit` so that they receive the graphs.

Maybe the best way is to save graph information in self?

But we actually can generalize the current code! Some lines are repeated and can be looped, assumptions can be dropped and differing code factored into functions. Loop over the zips to select train/val, and pass those to fit. Fit then doesn't assume the shapes of the zips and passes them onto `_train_step` and `_val_step`. We can probably generalize both functions into something as well!

After that it's just a matter of thinking how to inject this behavior in the pipeline. Maybe a common method and wrappers that select a class..

In [2]:
thesis = ThesisPipeline()
thesis.load_data()
thesis.make_protein_groups()
thesis.check_protein_groups()
thesis.remove_interior_nodes()
thesis.make_features()
thesis.make_adjacency_matrices()
thesis.make_targets()
thesis.convert_matrices_32bits()
#thesis.filter_by_node_amount(amount=1000)

thesis.make_neighborhoods(dist=3, negative_prob=0.5, verbose=1)
good_masks = thesis.all_masks
thesis.make_positive_weight()
thesis.make_laplacians()

thesis.pad_matrices()
#thesis.make_masks(pad_only=True)
thesis.make_masks()

INFO:root:There are 1proteins without group
INFO:root:Nodes w/o CDHit group: 247 (0.02368611581263995%)
By chain:{'5A99_A': 247}
INFO:root:Generating neighborhoods..
100% (1228 of 1228) |####################| Elapsed Time: 0:03:47 Time:  0:03:47
INFO:root:New class balance is 0.045730
INFO:root:For every non-contact point there are 0.04573015127697364 contact points (positive class weight = 21.867410714285715)
INFO:root:Maximum number of nodes per graph: 92


In [29]:
import gcn.gcn
reload(gcn.gcn)
thesis.GraphConvolutionalNetwork = gcn.gcn.GraphConvolutionalNetwork
LocalGCN = gcn.gcn.LocalGCN
thesis.LocalGCN = gcn.gcn.LocalGCN

In [31]:
import tensorflow as tf
import numpy as np
epochs=40
tf.data.Dataset.from_tensor_slices = lambda a: a
feats = thesis.all_features
supps = [tf.sparse.SparseTensor(indices, values.astype(np.float32), dense_shape)
            for indices, values, dense_shape in thesis.all_laplacians]
targs = thesis.all_targets
masks = thesis.all_masks
last_neighborhood = thesis.last_neighborhood
model = LocalGCN(feats[0].shape, 1, thesis.all_laplacians[0][2])
model.fit_cv_groups((feats, targs, supps, masks, last_neighborhood), thesis.protein_groups,
           positive_weight=thesis.fair_positive_weight, epochs=epochs)

N/A% (0 of 163921) |                     | Elapsed Time: 0:00:00 ETA:  --:--:--

Resetting weights..
Start of epoch 0


100% (163921 of 163921) |################| Elapsed Time: 0:07:50 Time:  0:07:50
  0% (35 of 163921) |                    | Elapsed Time: 0:00:00 ETA:   0:08:32

Epoch 0:
	TRAIN loss 0.54, auc 0.56, recall 0.70, precision 0.05
	VAL loss 0.57 auc 0.54 recall 0.68 precision 0.06
Confusion matrix(TRAIN):
[[2249277 3181924]
 [  75701  174119]]
Confusion matrix(VAL):
[[527987 805699]
 [ 22578  47589]]
Start of epoch 1


100% (163921 of 163921) |################| Elapsed Time: 0:07:42 Time:  0:07:42
  0% (35 of 163921) |                    | Elapsed Time: 0:00:00 ETA:   0:09:17

...

Epoch 37:
	TRAIN loss 0.51, auc 0.67, recall 0.76, precision 0.06
	VAL loss 0.58 auc 0.68 recall 0.70 precision 0.08
Confusion matrix(TRAIN):
[[2518261 2911795]
 [  58409  190066]]
Confusion matrix(VAL):
[[758411 576420]
 [ 21660  49852]]
Start of epoch 38


100% (163883 of 163883) |################| Elapsed Time: 0:07:42 Time:  0:07:42
  0% (91 of 163883) |                    | Elapsed Time: 0:00:00 ETA:   0:03:01

Epoch 38:
	TRAIN loss 0.51, auc 0.66, recall 0.77, precision 0.06
	VAL loss 0.58 auc 0.68 recall 0.71 precision 0.08
Confusion matrix(TRAIN):
[[2471156 2958900]
 [  56862  191613]]
Confusion matrix(VAL):
[[748554 586277]
 [ 21094  50418]]
Start of epoch 39


100% (163883 of 163883) |################| Elapsed Time: 0:07:38 Time:  0:07:38


Epoch 39:
	TRAIN loss 0.51, auc 0.67, recall 0.77, precision 0.06
	VAL loss 0.58 auc 0.68 recall 0.70 precision 0.08
Confusion matrix(TRAIN):
[[2498597 2931459]
 [  57410  191065]]
Confusion matrix(VAL):
[[750996 583835]
 [ 21192  50320]]
Best score is 0.7105663418769836, loading weights saved in 'best.h5


In [17]:
def run_cv_local_gcn(self, epochs=40):
    """Prepare tensors and run LocalGCN with Group K-Fold CV"""
    tf.data.Dataset.from_tensor_slices = lambda a: a
    feats = self.all_features
    supps = [tf.sparse.SparseTensor(indices, values.astype(np.float32), dense_shape)
                for indices, values, dense_shape in self.all_laplacians]
    targs = self.all_targets
    masks = self.all_masks
    last_neighborhood = self.last_neighborhood
    model = LocalGCN(feats[0].shape, 1, self.all_laplacians[0][2])
    model.fit_cv_groups((feats, targs, supps, masks, last_neighborhood), self.protein_groups,
               positive_weight=self.fair_positive_weight, epochs=epochs)
    return model
import types

thesis.run_cv_local_gcn = types.MethodType(ThesisPipeline.run_cv_local_gcn, thesis)

In [18]:
thesis.run_cv_local_gcn(epochs=2)

TypeError: super(type, obj): obj must be an instance or subtype of type

In [ ]:
import types
thesis.run_cv_gcn = types.MethodType(ThesisPipeline.run_cv_gcn, thesis)

In [ ]:
thesis.graphs[0].nodes[205]

In [ ]:
thesis.graphs[0].nodes[433]

In [ ]:
len(thesis.all_masks[0])

In [ ]:
import numpy as np
np.arange(10)

In [ ]:
import matplotlib.pyplot as plt
ones = np.ones_like(thesis.all_masks[0])
sum_all_masks = sum([np.minimum(mask, ones) for mask in thesis.all_masks])
plt.bar(np.arange(sum_all_masks.shape[0]), sum_all_masks)

In [ ]:
sum_all_targets = sum(thesis.all_targets)
plt.bar(np.arange(sum_all_targets.shape[0]), sum_all_targets/max(sum_all_targets))


In [ ]:
import matplotlib.pyplot as plt
sum_all_masks = sum(thesis.all_masks)
plt.bar(np.arange(sum_all_masks.shape[0]), sum_all_masks)

In [ ]:
thesis.run_cv_gcn(epochs=40)

In [ ]:
import gcn.gcn
import numpy as np
import tensorflow as tf
reload(gcn.gcn)
epochs = 40
tf.data.Dataset.from_tensor_slices = lambda a: a
feats = thesis.all_features
supps = [tf.sparse.SparseTensor(indices, values.astype(np.float32), dense_shape)
            for indices, values, dense_shape in thesis.all_laplacians]
targs = thesis.all_targets
masks = thesis.all_masks
model = gcn.gcn.GraphConvolutionalNetwork(feats[0].shape, 1, thesis.all_laplacians[0][2])
model.fit_cv_groups((feats, targs, supps, masks), thesis.protein_groups,
           positive_weight=thesis.fair_positive_weight, epochs=epochs)


In [ ]:
import tensorflow as tf
y = thesis.all_targets[0]
mask = thesis.all_masks[0]

tf.multiply(mask, y*24 + (1-y))

In [ ]:
tf.multiply(thesis.all_masks[6], thesis.all_targets[6]*24 + (1-thesis.all_targets[6]))

In [ ]:
import gcn
import tensorflow as tf
import numpy as np
"""Prepare tensors and run GCN with Group K-Fold CV"""
tf.data.Dataset.from_tensor_slices = lambda a: a
feats = thesis.all_features
supps = [tf.sparse.SparseTensor(indices, values.astype(np.float32), dense_shape)
            for indices, values, dense_shape in thesis.all_laplacians]
targs = thesis.all_targets
masks = thesis.all_masks
model = gcn.gcn.GraphConvolutionalNetwork(feats[0].shape, 1, thesis.all_laplacians[0][2])
model.fit_cv_groups((feats, supps, targs, masks), thesis.protein_groups,
           positive_weight=thesis.fair_positive_weight, epochs=epochs)

In [ ]:
import importlib
importlib.reload(gcn.gcn)

In [ ]:
for i, (mask, n) in enumerate(zip(thesis.all_masks, thesis.nb_nodes_per_graph)):
    thesis.all_masks[i] = np.pad(mask, (0, thesis.nb_nodes - n)).astype(np.float32)

In [ ]:
masks[0].shape[0]

In [ ]:
epochs=40
masks = thesis.all_masks
assert min([m.shape[0] for m in masks]) == max([m.shape[0] for m in masks])
model = gcn.gcn.GraphConvolutionalNetwork(feats[0].shape, 1, thesis.all_laplacians[0][2])
model.fit_cv_groups((feats, supps, targs, masks), thesis.protein_groups,
           positive_weight=thesis.fair_positive_weight, epochs=epochs)

In [ ]:
thesis.all_masks[2]

In [ ]:
thesis.pad_matrices()

thesis.run_cv_gcn(epochs=40)

In [ ]:
thesis.all_features[0].shape

### XGBoost model

In [ ]:
thesis = ThesisPipeline()
thesis.load_data() # ok
thesis.make_protein_groups() # ok probably..?
thesis.check_protein_groups() # same
thesis.remove_interior_nodes() # ok
# get diffused graph
# graph 2 dataframe, target, drop chain
thesis.propagate_features_graph(steps=3)
thesis.make_diffused_features() #definitely not
thesis.make_targets() # we can probably use this
thesis.filter_by_node_amount(amount=1000) #we don't use it dude => we should, for consistency!
#assign groups, concatenate dfs
thesis.run_cv_hypersearch_xgb()

## Static contact graph

### GCN

In [ ]:
thesis = ThesisPipeline()
thesis.load_data(graphs_folder="../graphs/contacts/")
thesis.filter_graphs_missing_data()
thesis.make_protein_groups()
thesis.check_protein_groups()
thesis.make_features()
thesis.make_adjacency_matrices()
thesis.make_targets()
thesis.convert_matrices_32bits()
thesis.filter_by_node_amount(amount=1000)
thesis.make_positive_weight()
thesis.make_laplacians()
thesis.pad_matrices()
thesis.make_masks()
thesis.run_cv_gcn(epochs=40)

### XGBoost

In [ ]:
4

In [ ]:
#357/814

In [ ]:
thesis.run_cv_hypersearch_xgb(3)

In [ ]:
for i, graph in enumerate(thesis.graphs):
    _df = biograph.graph_models.GraphModel.graph_to_dataframe(graph)
    if len(_df.select_dtypes(np.object).columns) > 3:
        break
i

In [ ]:
G = graph.copy()
all_cols = set()
for node_idx in G.nodes:
    nice_dict = {k:v for k, v in G.nodes[node_idx].items() if "_" not in k}
    G.nodes[node_idx].clear()
    G.nodes[node_idx].update(nice_dict)
    for col in G.nodes[node_idx].keys():
        all_cols.add(col)
all_cols

In [ ]:
import biograph.graph_models

In [ ]:
G_df = biograph.graph_models.GraphModel.graph_to_dataframe(G)
diffused_G = biograph.graph_models.GraphModel.get_diffused_graph(G, steps = 3)
diffused_G_df = biograph.graph_models.GraphModel.graph_to_dataframe(diffused_G)

In [ ]:
G_df.dtypes

In [ ]:
diffused_G_df.dtypes

In [ ]:
class GraphModel:
    def __init__(self):
        self.G = nx.Graph()
    def generate_graph(self, protein, params):
        raise NotImplementedError
    def add_features(self, dataframe, columns):
        raise NotImplementedError
    @staticmethod
    def graph_to_dataframe(G):
        return pd.DataFrame.from_dict({node_idx: {feature: val for feature,val in G.nodes[node_idx].items()}
              for node_idx in G.nodes}, orient="index")
    @staticmethod
    def get_diffused_graph(G, aggregator = None, keys = None, steps = 1):
        """Diffuses some `keys` features across graph neighbors that are
        up to `steps` apart and afterwards processes the groups using
        `aggregator`.
        Diffused features end with "_n" with n being the steps to the node.

        Parameters
        ----------
        aggregator: function or None
            A function that receives a dictionary of list of diffused
            features (e.g. {mass_1 => [1,12], mass_2 => [8,1,13]}) and
            returns a new dictionary.
            If None, non-numeric (i.e. not complex, int, float or bool)
            features are discarded and numeric features are averaged.
        keys: list
            Features to be diffused. All nodes in graph must have them.
        steps: int
            Number of steps from each node to be used in diffusion.

        Returns
        -------
        networkx.Graph with same structure as before but with additional
        diffused features.
        """
        if keys is None:
            some_node = list(G.nodes)[0]
            keys = G.nodes[some_node].keys()

        diffused_graph = G.copy()
        # Setup feature holders.
        for node_idx in diffused_graph.nodes:
            for key in keys:
                for dist in range(1, steps+1):
                    diffused_graph.nodes[node_idx]["{}_{}".format(key, dist)] = list()

        new_keys = set()
        for node_idx in G.nodes:
            # nx doesn't offer a bfs traversal that also yields distances,
            # so we need to keep track of them.
            distances = {node_idx: 0}
            for origin, neighbor in nx.bfs_edges(G, node_idx):
                distances[neighbor] = distances[origin] + 1
                if distances[neighbor] > steps:
                    break
                for key in keys:
                    neighbor_value = diffused_graph.nodes[neighbor][key]
                    diffused_key = "{}_{}".format(key, distances[neighbor])
                    new_keys.add(diffused_key)
                    try:
                        diffused_graph.nodes[node_idx][diffused_key].append(neighbor_value)
                    except KeyError:
                        print(neighbor_value)
                        print(keys)
                        print(diffused_key)
                        print(diffused_graph.nodes[node_idx])
                        return

        # Process feature lists with an aggregator.
        for node_idx in diffused_graph.nodes:
            node_features = diffused_graph.nodes[node_idx].copy()

            diffused_features = {key:val
                for key, val in node_features.items()
                if key in new_keys}

            if aggregator is None:
                # Basic aggregator. If the type is numeric-like then do a mean, otherwise
                # discard it.
                diffused_features = {k:v for k,v in diffused_features.items()
                    if len(v)>0 and isinstance(v[0], (int, float, complex))}
                for diffused_key in diffused_features.keys():
                    feature_list = diffused_features[diffused_key]
                    diffused_features[diffused_key] = sum(feature_list)/len(feature_list)

            else:
                diffused_features = aggregator(diffused_features)

            # We can't replace the dict object, so here:
            diffused_graph.nodes[node_idx].clear()
            diffused_graph.nodes[node_idx].update(diffused_features)

            # Add back the original node-specific values ("distance zero" and ignored
            # columns).
            for key in node_features.keys():
                if key not in new_keys:
                    diffused_graph.nodes[node_idx][key] = node_features[key]

        return diffused_graph

In [ ]:
new_keys

In [ ]:
{key:val
        for key, val in node_features.items()
        if key in new_keys}.keys()

In [ ]:
diffused_features.keys

In [ ]:
diffused_features.keys()

In [ ]:
len(thesis.all_features), len(thesis.graphs)

In [ ]:
for i, df in enumerate(thesis.all_features):
    selected = df.select_dtypes(np.object)
    if selected.shape[1] > 0:
        print(i)
        break
        if [col for col in selected.columns if "_" not in col]:
            break
    if i == len(thesis.all_features)-1:
        df = None

In [ ]:
df.select_dtypes(np.object)

In [ ]:
thesis.all_features[0].select_dtypes()

In [ ]:

import os
import logging
import subprocess
import progressbar
import networkx as nx
import numpy as np
import scipy.sparse as sp
import pandas as pd
import tensorflow as tf
from gcn.gcn import Laplacian, sparse_to_tuple, GraphConvolutionalNetwork

def make_diffused_features(self, steps=3):
    """Obtain diffused features from already diffused graphs"""
    from biograph.graph_models import GraphModel
    import biograph.constants

    self.all_features = []
    logging.info("Generating features..")
    for graph in progressbar.progressbar(self.graphs):
        df = GraphModel.graph_to_dataframe(graph)
        # One hot for AAs
        for code3 in biograph.constants.AMINOACIDS_3 + ["UNK"]:
            df[code3] = (df.resname == code3).astype(np.int)

        if "coord" in df.columns:
            for i, coord in enumerate(["x", "y", "z"]):
                df[coord] = df.coord.map(lambda x: x[i])

        df = df[[c for c in df.columns if "distance_" not in c]]
        df = df.drop(["full_id","resname", "coord", "distance", "chain"], axis=1, errors="ignore")
        if not self.all_features:
            logging.info(f"Dataframe features: {df.columns}")
        self.all_features.append(df)

In [ ]:
def run_cv_hypersearch_xgb(self, n_iter):
    """Concatenate all dataframes and run XGB with Group K-Fold CV
    and Randomized Hyperparametr Search."""
    from scipy.stats import uniform
    import sklearn.model_selection
    import xgboost as xgb

    for df, group, target in zip(self.all_features, self.protein_groups, self.all_targets):
        df["target"] = target
        df["group"] = group
    dataset = pd.concat(self.all_features, ignore_index=True)
    dataset = dataset.loc[dataset.group.notna()]

    row_groups = dataset.group
    row_target = dataset.target
    dataset = dataset.drop(["group", "target"], axis=1)

    logging.info(f"Dataframe shape: {dataset.shape}")
    logging.info(f"Columns passed to model: {dataset.columns}")
    logging.info(f"Data types: {dataset.dtypes}")
    param = {'max_depth': 2, 'eta': 1, 'objective': 'binary:logistic',
             'nthread': 4, 'eval_metric': 'auc'}

    groupk= sklearn.model_selection.GroupKFold(n_splits=5)
    clf = xgb.XGBClassifier(**param)

    clf = sklearn.model_selection.RandomizedSearchCV(
        clf, {"max_depth": list(range(3,10)),
            "min_child_weight": uniform(loc=0.5, scale=1.5), # ~U(loc, loc+scale)
            "eta": uniform(loc=0.2, scale=0.3)},
        cv=groupk, n_iter=n_iter, scoring="roc_auc", verbose=5, n_jobs=3)

    search = clf.fit(dataset, row_target, row_groups)

    return clf, search

In [ ]:
import types
#thesis.make_diffused_features = types.MethodType(make_diffused_features, thesis)
thesis.run_cv_hypersearch_xgb = types.MethodType(run_cv_hypersearch_xgb, thesis)